In [13]:
import matplotlib.pyplot as plot
import matplotlib.patches as patches
import matplotlib.patches as Rectangle
from PIL import Image
import os, cv2
import csv
import math
import numpy as np
import copy
import operator

In [14]:
# init :  image_path를 받고 img_list에 파일들 리스트를 작성함.
# run : csv 파일을 불러온후 파일이름 : [[x_y_w_h],[x_y_w_h]] 형태의 딕셔너리를 만듬.

class MakeListGT():
    def __init__(self):
        pass
    def get_img_list(self, image_path):
        self.img_list = sorted(os.listdir(image_path))
        return self.img_list
    
    def get_gt_dict(self, CSV_path):
        file = open(CSV_path,'r') 
        data = {}
        with file:
            reader = csv.reader(file, delimiter = "\t") 
            for k,row in enumerate(reader):
                if k==0:
                    continue
                row_sep = row[0].split(',')    #row[0] 으로 굳이 하는 이유 : row 그 자체는 list이기 때문에 split 함수가 없음. 따라서 구체적으로 list안의 내용물 ' str ' 을  가리켜서 split 함수를 사용
                image_name = row_sep[-1].split('/')[-1]  #str 
                x_y_w_h = row_sep[2:6]      #list
                if (image_name in data) ==False : 
                    x_y_w_h_list =[]
                    x_y_w_h_list.append(x_y_w_h)
                    data[image_name] = x_y_w_h_list
                else:
                    data[image_name].append(x_y_w_h)
       
        file.close()
        return data

In [19]:
def area_xywh(img_gt_list,img_shape):  # 인풋들이 [[x,y,w,h], [x,y,w,h]] 형태의 list가 들어오면 [[박스크기,x,y,w,h],[박스크기,x,y,w,h]] 로 반환
    
    area_xywh_list = []
    for k,xywh in enumerate(img_gt_list):
        if k == 0 :
            print("xywh =" , xywh)
        quadrangle = (xywh[0]+xywh[2])*(img_shape[1]-xywh[1])   # (x+w) * (img_height - y)
        area_xywh_list.append([quadrangle, xywh])
        
    
    return area_xywh_list

In [20]:
def make_group(area_xywh_list): 
    sorted_area_xywh_list = sorted(area_xywh_list, reverse=True)  #사각형 크기대로 줄서기
    
    mid = sorted_area_xywh_list[0][1][0]+sorted_area_xywh_list[0][1][2]/2    #[0][1][0] = x , [0][1][2] = w
    revision_factor = int(0.3*sorted_area_xywh_list[0][1][2]/2)
    group_list = []
    not_group_list = []
    
    for k,j in enumerate(sorted_area_xywh_list):
        if (j[1][0] + j[1][2] > (mid - revision_factor)) == (j[1][0] < (mid + revision_factor) ):
            group_list.append(j[1])
        else :
            not_group_list.append(j)
    
    return sorted(group_list, key=operator.itemgetter(1)), sorted(not_group_list)

In [21]:
def numbering(input_path,output_path,csv_path):   # invoke def MakeListGT, def make_group, def area_xywh
    
    mlgt = MakeListGT()
    gt_dict = mlgt.get_gt_dict(csv_path)
    
    
    for k,(img_name, img_gt_list) in enumerate(gt_dict.items()): #모든 이미지 순환문
        
        img = cv2.imread(input_path +"/"+ img_name)
        print("이미지. shape  = ",  img.shape)
        
        #------------------------------------------------------------ 요소들 str  to int 값 변환 -------------------------------------------------------
        for k,j in enumerate(img_gt_list):  #해당 이미지의 [[gt],[gt],[gt],[gt],...] 형태의 gt 리스트 모음. 
            img_gt_list[k] = [int(i) for i in j]  
        #-------------------------------------------------------------------------------------------------------------------------------------------------
        
        area_xywh_list = area_xywh(img_gt_list,img.shape) #외부 함수 호출. [사각형 크기, [ x,y,w,h 좌표], ...] 의 형태로 반환
        group_dict = {}
        tem_group_list =[]
        dict_num = 0
        
        while True:
            
            group_list , area_xywh_list = make_group(area_xywh_list) #외부 함수 호출. 자세한 설명은 문서 참조. # area_xywh_list = not_group_list 
            group_dict[dict_num] = group_list
            
    
            dict_num += 1
            if len(area_xywh_list) == 0 :
                break
        
        print("key 값과 value 들  = " ,group_dict)
        order_key_list = []  # 무리 그룹들 각각의 x값을 기준으로 다시 재정렬한 key 값을 모은 리스트. (ex : 본래 key 예 0, 1, 2, 순서대로 일때 본래 x 재정렬 하면 0, 16, 1 등...
        for j in sorted(group_dict.items(), key =operator.itemgetter(1) , reverse=True):
            
            order_key_list.append(j[0])
        
        
        #------------------------------------------------------------putText 로 실제 숫자 그리기 ---------------------------------------------------------------
        real_num = 0
        for j in order_key_list:  # j  = key number 
            color = list(np.random.random(size=3) * 255)
            #cv2.line(img,(group_dict[j][0][0]+int(group_dict[j][0][2]/2),group_dict[j][0][1]+int(group_dict[j][0][3]/2)),(group_dict[j][-1][0]+int(group_dict[j][-1][2]/2),group_dict[j][-1][1]+int(group_dict[j][-1][3]/2)),color,5)
            
            for i,x_y_w_h in enumerate(group_dict[j]):
                
                x,y,w,h = x_y_w_h[0], x_y_w_h[1], x_y_w_h[2], x_y_w_h[3]
                
                if i == 0:
                    cv2.putText(img, str(j),(int(x+w*2/3),int(y+h/2)),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,2,(0,0,255),2,cv2.LINE_AA)
                    
                if img.shape[0]*img.shape[1] > 500000 :
                    scalefactor =1
                    thickness = 2
                elif img.shape[0]*img.shape[1] > 400000 :
                    scalefactor = 0.8
                    thickness = 1
                elif img.shape[0]*img.shape[1] > 300000:
                    scalefactor = 0.3
                    thickness = 1
                else :
                    scalefactor = 0.3
                    thickness = 1
                
                cv2.rectangle(img,(x,y),(x+w,y+h) , color , 2)
                cv2.putText(img, str(real_num), (int(x+w/5),int(y+2*h/3)),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, scalefactor,(0,255,0), thickness, cv2.LINE_AA)
                real_num += 1
        cv2.imwrite(output_path + "/" + img_name, img)
    
        
        
        
        
        

In [22]:

input_path = "/home/ihs/Documents/프로젝트/hanja_project/CRC_data/사진파일"
output_path = "/home/ihs/Documents/프로젝트/hanja_project/CRC_data/numbering"
csv_path = "/home/ihs/Documents/프로젝트/hanja_project/CRC_data/RecognitionmodelDataset_utf.csv"

import shutil
shutil.rmtree(r"/home/ihs/Documents/프로젝트/hanja_project/CRC_data/numbering")
os.mkdir("/home/ihs/Documents/프로젝트/hanja_project/CRC_data/numbering/")
numbering(input_path, output_path, csv_path)

이미지. shape  =  (2267, 1287, 3)
xywh = [1144, 213, 73, 85]
key 값과 value 들  =  {0: [[1144, 213, 73, 85], [1147, 291, 73, 63], [1141, 349, 80, 87], [1147, 424, 98, 69], [1159, 493, 58, 68], [1139, 551, 91, 98], [1147, 646, 78, 69], [1149, 704, 90, 107], [1147, 790, 87, 84], [1141, 873, 84, 73], [1148, 936, 92, 74], [1149, 993, 88, 72], [1158, 1049, 74, 93], [1155, 1127, 80, 99], [1166, 1214, 47, 41], [1157, 1249, 86, 96], [1161, 1327, 82, 60], [1155, 1384, 88, 78], [1161, 1464, 77, 88], [1158, 1536, 84, 100], [1173, 1624, 85, 82], [1165, 1690, 80, 62], [1165, 1734, 86, 72], [1166, 1790, 78, 82], [1163, 1856, 82, 78], [1179, 1924, 57, 53], [1179, 1958, 59, 95]], 1: [[1042, 210, 97, 73], [1049, 271, 85, 108], [1054, 367, 88, 51], [1072, 413, 45, 72], [1061, 476, 64, 90], [1051, 549, 87, 84], [1062, 621, 69, 81], [1071, 685, 61, 70], [1059, 743, 76, 86], [1061, 820, 80, 82], [1062, 880, 80, 93], [1073, 952, 68, 90], [1062, 1046, 86, 86], [1077, 1114, 76, 91], [1087, 1182, 51, 74], [1071, 124

이미지. shape  =  (2267, 1324, 3)
xywh = [1211, 233, 95, 86]
key 값과 value 들  =  {0: [[1211, 233, 95, 86], [1213, 314, 97, 82], [1202, 392, 108, 100], [1201, 482, 102, 86], [1216, 557, 87, 92], [1206, 639, 94, 82], [1221, 721, 60, 59], [1194, 776, 84, 85], [1194, 858, 90, 72], [1198, 918, 85, 89], [1201, 1002, 68, 81], [1195, 1071, 86, 69], [1198, 1143, 64, 97], [1196, 1220, 87, 89], [1183, 1304, 85, 97], [1194, 1395, 80, 61], [1189, 1445, 76, 97], [1180, 1543, 88, 78], [1186, 1624, 91, 66], [1190, 1689, 79, 54], [1178, 1736, 81, 110], [1177, 1836, 95, 113], [1179, 1947, 100, 62], [1181, 1995, 92, 83]], 1: [[1101, 232, 110, 95], [1104, 311, 108, 112], [1104, 405, 88, 88], [1113, 482, 81, 85], [1110, 566, 93, 83], [1099, 638, 111, 80], [1102, 717, 108, 57], [1112, 776, 72, 86], [1094, 864, 103, 54], [1100, 908, 80, 81], [1096, 986, 85, 54], [1094, 1036, 69, 65], [1102, 1093, 71, 71], [1088, 1158, 97, 85], [1093, 1240, 90, 52], [1089, 1289, 85, 92], [1091, 1367, 89, 91], [1077, 1452, 101, 99

이미지. shape  =  (2267, 1299, 3)
xywh = [1144, 244, 99, 97]
key 값과 value 들  =  {0: [[1144, 244, 99, 97], [1151, 335, 87, 91], [1151, 411, 96, 71], [1155, 473, 100, 104], [1160, 560, 87, 94], [1159, 636, 100, 84], [1177, 713, 75, 63], [1170, 763, 87, 107], [1164, 852, 91, 99], [1165, 933, 95, 93], [1180, 1011, 67, 90], [1167, 1084, 101, 84], [1169, 1161, 82, 101], [1171, 1251, 96, 73], [1172, 1318, 86, 81], [1174, 1389, 73, 91], [1167, 1468, 103, 65], [1177, 1533, 89, 84], [1176, 1598, 83, 102], [1189, 1689, 68, 72], [1167, 1761, 96, 81], [1173, 1833, 93, 79], [1168, 1912, 80, 84], [1170, 1978, 81, 75], [1176, 2042, 94, 66]], 1: [[1064, 255, 85, 93], [1047, 336, 109, 109], [1060, 426, 84, 76], [1055, 474, 103, 100], [1065, 576, 94, 57], [1070, 613, 100, 101], [1070, 708, 76, 84], [1073, 782, 92, 78], [1079, 846, 83, 96], [1091, 933, 71, 94], [1090, 1015, 82, 86], [1087, 1095, 90, 85], [1086, 1179, 89, 69], [1102, 1239, 63, 79], [1081, 1308, 90, 98], [1083, 1390, 88, 94], [1078, 1465, 83, 

이미지. shape  =  (2267, 1318, 3)
xywh = [1208, 243, 81, 89]
key 값과 value 들  =  {0: [[1208, 243, 81, 89], [1203, 330, 84, 87], [1204, 413, 77, 54], [1210, 462, 70, 81], [1189, 547, 99, 76], [1206, 609, 76, 75], [1195, 680, 90, 75], [1212, 750, 50, 52], [1190, 805, 82, 89], [1208, 897, 48, 42], [1191, 932, 79, 73], [1199, 1003, 71, 83], [1199, 1083, 56, 62], [1195, 1147, 75, 80], [1192, 1223, 70, 89], [1194, 1310, 51, 57], [1192, 1374, 59, 61], [1186, 1438, 80, 70], [1188, 1508, 66, 62], [1184, 1568, 77, 75], [1175, 1648, 95, 52], [1184, 1700, 85, 75], [1195, 1766, 56, 83], [1186, 1846, 79, 64], [1187, 1903, 78, 68], [1190, 1962, 71, 92]], 1: [[1121, 238, 71, 88], [1118, 319, 82, 88], [1124, 405, 67, 76], [1111, 476, 85, 77], [1110, 545, 75, 79], [1108, 626, 82, 67], [1113, 693, 65, 91], [1106, 784, 80, 67], [1106, 839, 49, 83], [1112, 924, 46, 43], [1087, 967, 73, 84], [1097, 1056, 73, 69], [1100, 1118, 67, 54], [1101, 1170, 64, 74], [1094, 1253, 66, 67], [1104, 1315, 62, 65], [1096, 1380

이미지. shape  =  (775, 959, 3)
xywh = [864, 35, 52, 37]
key 값과 value 들  =  {0: [[864, 35, 52, 37], [862, 68, 56, 41], [867, 108, 51, 28], [869, 134, 47, 22], [862, 155, 51, 35], [869, 191, 44, 36], [871, 227, 36, 23], [864, 249, 42, 30], [868, 279, 45, 31], [864, 313, 42, 20], [864, 334, 50, 29], [871, 360, 30, 25], [861, 384, 50, 36], [865, 420, 43, 29], [862, 446, 45, 38], [865, 483, 42, 38], [862, 520, 46, 39], [861, 556, 52, 39], [861, 596, 45, 32], [861, 640, 50, 26], [861, 665, 46, 25], [873, 694, 34, 32]], 1: [[811, 40, 50, 46], [816, 83, 45, 35], [821, 117, 37, 31], [818, 146, 40, 45], [814, 188, 53, 33], [814, 220, 46, 37], [815, 257, 46, 29], [815, 288, 47, 32], [816, 320, 44, 36], [816, 355, 41, 29], [815, 384, 43, 32], [812, 415, 48, 35], [818, 451, 39, 29], [815, 479, 42, 24], [816, 501, 42, 42], [815, 543, 42, 34], [812, 576, 46, 27], [814, 602, 44, 32], [816, 634, 39, 29], [814, 663, 43, 28], [816, 690, 46, 31]], 2: [[766, 40, 45, 30], [770, 70, 38, 31], [772, 101, 39, 33]

...

이미지. shape  =  (1431, 741, 3)
xywh = [659, 103, 57, 89]
key 값과 value 들  =  {0: [[659, 103, 57, 89], [656, 190, 68, 89], [661, 275, 64, 53], [658, 320, 68, 63], [657, 375, 63, 84], [660, 503, 60, 84], [656, 585, 72, 92], [652, 671, 77, 84], [649, 743, 83, 75], [649, 815, 76, 88], [658, 897, 72, 70], [644, 956, 92, 98], [652, 1050, 78, 64], [654, 1111, 75, 64], [658, 1171, 71, 73], [660, 1237, 66, 64], [662, 1298, 65, 49], [650, 1345, 74, 77]], 1: [[558, 48, 87, 105], [563, 142, 77, 57], [559, 195, 82, 64], [552, 259, 87, 73], [551, 318, 96, 91], [554, 399, 89, 80], [557, 472, 83, 92], [556, 570, 84, 82], [544, 641, 87, 98], [542, 732, 77, 74], [550, 795, 81, 68], [554, 865, 80, 90], [564, 955, 68, 79], [564, 1024, 61, 46], [563, 1063, 60, 68], [562, 1130, 72, 77], [560, 1199, 71, 58], [558, 1254, 82, 93], [564, 1340, 70, 61]], 2: [[478, 117, 72, 121], [476, 229, 67, 96], [477, 320, 66, 95], [465, 405, 81, 92], [464, 484, 87, 87], [468, 563, 70, 71], [465, 627, 70, 52], [449, 667, 85, 10

이미지. shape  =  (1431, 742, 3)
xywh = [685, 28, 53, 63]
key 값과 value 들  =  {0: [[685, 28, 53, 63], [688, 79, 53, 66], [688, 141, 49, 69], [698, 205, 38, 58], [691, 255, 47, 65], [688, 317, 52, 47], [688, 357, 51, 50], [685, 403, 53, 58], [685, 456, 54, 52], [686, 504, 55, 62], [686, 568, 53, 69], [682, 630, 51, 72], [683, 702, 53, 76], [686, 776, 55, 77], [679, 851, 59, 77], [679, 927, 59, 81], [684, 998, 54, 53], [686, 1046, 56, 90], [690, 1125, 50, 74], [692, 1189, 50, 75], [698, 1259, 39, 46], [701, 1301, 42, 69], [701, 1367, 43, 61]], 1: [[617, 16, 73, 89], [630, 98, 56, 83], [622, 172, 61, 65], [615, 226, 69, 78], [620, 300, 68, 65], [622, 356, 57, 47], [623, 393, 62, 67], [617, 452, 67, 64], [618, 502, 60, 56], [615, 551, 64, 73], [621, 620, 61, 65], [612, 676, 63, 76], [610, 737, 78, 59], [609, 783, 72, 59], [613, 835, 68, 44], [595, 870, 83, 100], [606, 963, 78, 73], [611, 1029, 73, 93], [602, 1115, 94, 70], [614, 1174, 77, 91], [618, 1259, 75, 65], [624, 1310, 76, 63], [626, 13

이미지. shape  =  (1424, 746, 3)
xywh = [644, 58, 94, 113]
key 값과 value 들  =  {0: [[644, 58, 94, 113], [656, 162, 78, 89], [659, 253, 75, 80], [650, 321, 89, 139]], 1: [[524, 59, 102, 136], [532, 191, 93, 114], [525, 305, 91, 98], [539, 405, 95, 110], [536, 522, 94, 94], [533, 617, 109, 92], [540, 697, 94, 105], [533, 790, 88, 103], [533, 887, 100, 52], [530, 929, 101, 98], [533, 1015, 93, 92], [522, 1102, 101, 110], [536, 1202, 80, 74], [522, 1270, 91, 128]], 2: [[435, 101, 90, 144], [429, 245, 87, 110], [432, 355, 73, 94], [484, 411, 55, 93], [422, 464, 77, 78], [410, 536, 101, 131], [429, 658, 87, 81], [411, 731, 88, 78], [422, 799, 88, 77], [411, 874, 104, 127], [406, 992, 108, 142], [413, 1122, 111, 98], [416, 1219, 91, 92], [407, 1307, 101, 88]], 3: [[363, 90, 72, 105], [351, 191, 81, 64], [336, 252, 96, 81], [337, 338, 86, 99], [333, 428, 80, 79], [327, 503, 85, 120], [334, 617, 75, 114], [326, 720, 83, 87], [310, 800, 104, 125], [315, 914, 84, 85], [315, 988, 80, 88], [303, 1210, 

이미지. shape  =  (1429, 739, 3)
xywh = [651, 64, 83, 107]
key 값과 value 들  =  {0: [[651, 64, 83, 107], [631, 163, 102, 101], [625, 260, 99, 103], [627, 371, 104, 119], [626, 478, 95, 120], [625, 599, 100, 97], [630, 689, 106, 91], [625, 780, 103, 82], [629, 854, 97, 140]], 1: [[516, 25, 122, 94], [526, 117, 93, 75], [505, 212, 115, 109], [504, 318, 104, 98], [494, 422, 131, 106], [482, 519, 132, 132], [495, 633, 119, 84], [493, 698, 114, 122], [508, 815, 98, 94], [511, 904, 70, 90], [498, 983, 96, 125], [480, 1103, 121, 112], [500, 1219, 94, 80], [503, 1280, 94, 72], [496, 1347, 103, 72]], 2: [[411, 70, 110, 90], [406, 141, 99, 95], [409, 233, 83, 89], [407, 321, 94, 87], [391, 408, 108, 105], [400, 502, 74, 94], [390, 586, 93, 103], [386, 676, 98, 115], [395, 783, 96, 92], [385, 875, 97, 122], [393, 1000, 78, 114], [393, 1123, 79, 94], [392, 1213, 99, 85], [376, 1301, 120, 100]], 3: [[304, 56, 117, 117], [303, 160, 94, 142], [296, 298, 84, 130], [274, 427, 115, 103], [281, 530, 108, 107]

이미지. shape  =  (892, 503, 3)
xywh = [454, 110, 18, 27]
key 값과 value 들  =  {0: [[454, 110, 18, 27], [451, 145, 24, 17], [450, 169, 30, 29], [454, 205, 24, 30], [456, 237, 26, 32], [449, 274, 40, 27], [453, 300, 32, 31], [458, 339, 25, 23], [462, 373, 19, 27], [454, 405, 30, 29], [459, 438, 29, 34], [461, 504, 25, 13], [464, 522, 19, 24], [463, 594, 24, 22], [462, 726, 28, 34]], 1: [[416, 110, 16, 30], [419, 148, 24, 22], [420, 180, 20, 15], [415, 205, 25, 27]], 2: [[381, 76, 14, 26], [385, 113, 12, 15], [385, 133, 11, 16], [382, 150, 19, 29], [383, 188, 20, 26], [385, 221, 21, 29], [380, 250, 22, 25], [383, 283, 23, 29], [376, 314, 29, 25], [379, 341, 21, 28], [378, 380, 24, 23], [380, 409, 18, 25], [375, 440, 22, 25], [380, 475, 21, 24], [380, 503, 22, 26], [375, 535, 25, 18], [381, 563, 22, 28], [379, 607, 24, 17], [377, 626, 28, 29], [378, 664, 20, 21], [377, 685, 22, 36], [381, 724, 22, 34], [374, 764, 27, 27], [376, 791, 30, 33]], 3: [[342, 119, 20, 31], [341, 162, 26, 23], [347, 1